<a href="https://colab.research.google.com/github/Jair2472/Proyecto_Prediccion_MargenBruto/blob/main/EDA_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 0. Configuración Visual ---
# Un estilo más limpio y profesional para los reportes de negocio.
plt.style.use('seaborn-v0_8-ticks')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['font.family'] = 'sans-serif'


# --- 1. Carga y Preparación de Datos ---
# Cargar los datos. Se asume que el archivo se llama 'dataset_carniceria.csv'
try:
    df = pd.read_csv('dataset_carniceria.csv')
except FileNotFoundError:
    print("ADVERTENCIA: No se encontró 'dataset_carniceria.csv'. Se usarán datos de ejemplo.")
    date_range = pd.to_datetime(pd.date_range(start='2022-01-01', periods=365*2, freq='D'))
    data = {
        'Date': date_range,
        'FiscalYear': date_range.year,
        'DiaSemana': date_range.dayofweek,
        'VentasAcum': (np.sin(np.arange(365*2) * 2 * np.pi / 30) * 5000 + np.sin(np.arange(365*2) * 2 * np.pi / 7) * 2000 + np.random.rand(365*2) * 1000 + 10000).cumsum(),
        'IngresosAcum': (np.sin(np.arange(365*2) * 2 * np.pi / 30) * 1000 + np.sin(np.arange(365*2) * 2 * np.pi / 7) * 400 + np.random.rand(365*2) * 200 + 2000).cumsum(),
    }
    df = pd.DataFrame(data)

# Convertir la columna 'Date' a formato datetime.
if isinstance(df['Date'].iloc[0], str):
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Ordenar el dataframe y calcular valores DIARIOS (el paso más importante)
df = df.sort_values('Date').reset_index(drop=True)
df['VentasDiarias'] = df['VentasAcum'].diff().fillna(df['VentasAcum'].iloc[0])
df['IngresosDiarios'] = df['IngresosAcum'].diff().fillna(df['IngresosAcum'].iloc[0])
df['MargenDiario'] = (df['IngresosDiarios'] / (df['VentasDiarias'] + 1e-6)) # Se añade 1e-6 para evitar división por cero
df.loc[df['VentasDiarias'] <= 0, 'MargenDiario'] = 0 # Corregir márgenes anómalos en días sin venta
df['Año'] = df['Date'].dt.year
df['Mes'] = df['Date'].dt.month
df['DiaDelAño'] = df['Date'].dt.dayofyear

# --- HISTORIA 1: ¿VENDEMOS MUCHO O VENDEMOS BIEN? (Análisis de Margen vs. Volumen) ---
# Este gráfico nos ayuda a entender si los días de altas ventas también son los de mayor rentabilidad.
# Es clave para la estrategia de precios y promociones.
plt.figure(figsize=(16,9))
sns.scatterplot(
    data=df,
    x='VentasDiarias',
    y='MargenDiario',
    hue='Año',
    palette='viridis',
    alpha=0.6,
    size='IngresosDiarios', # El tamaño de la burbuja es el ingreso neto
    sizes=(20, 500)
)
plt.title('Relación entre Volumen de Venta y Rentabilidad Diaria', weight='bold')
plt.xlabel('Ventas Diarias ($)')
plt.ylabel('Margen de Ganancia Diario')
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:.1%}'.format)) # Formatear eje Y como porcentaje
plt.axhline(df['MargenDiario'].median(), color='red', linestyle='--', label=f'Mediana de Margen ({df["MargenDiario"].median():.1%})')
plt.legend(title='Año y Ganancia Neta')
plt.savefig('historia_accionable_1_margen_vs_volumen.png', bbox_inches='tight')
plt.close()


# --- HISTORIA 2: ANATOMÍA DE UN DÍA "EXCELENTE" vs UN DÍA "MALO" ---
# Clasificamos cada día según su rendimiento en ventas para entender qué patrones siguen los mejores (y peores) días.
categorias = ['Malo', 'Regular', 'Bueno', 'Excelente']
df['Rendimiento'] = pd.qcut(df['VentasDiarias'], 4, labels=categorias)

# Mapear DiaSemana a nombres de días y ordenar
dias_map = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
df['NombreDia'] = df['DiaSemana'].map(dias_map)
orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

# Crear una tabla de contingencia y graficar
contingencia = pd.crosstab(df['NombreDia'], df['Rendimiento'], normalize='index').reindex(orden_dias) * 100
ax = contingencia.plot(kind='bar', stacked=True, figsize=(16, 9), colormap='RdYlGn')
plt.title('Composición del Rendimiento por Día de la Semana', weight='bold')
plt.ylabel('% de Ocurrencias')
plt.xlabel('Día de la Semana')
plt.xticks(rotation=0)
plt.legend(title='Rendimiento', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.9, 1])
plt.savefig('historia_accionable_2_anatomia_dias.png', bbox_inches='tight')
plt.close()


# --- HISTORIA 3: LA CARRERA ANUAL - ¿ESTAMOS SUPERANDO AL AÑO PASADO? ---
# Este gráfico muestra la acumulación de ventas a lo largo de cada año.
# Es la forma más directa de saber si el negocio está creciendo y a qué ritmo.
df_carrera = df.groupby(['Año', 'DiaDelAño'])['VentasDiarias'].sum().reset_index()
df_carrera['VentasAnualAcum'] = df_carrera.groupby('Año')['VentasDiarias'].cumsum()
palette = sns.color_palette("coolwarm", len(df_carrera['Año'].unique()))

plt.figure(figsize=(16,9))
sns.lineplot(data=df_carrera, x='DiaDelAño', y='VentasAnualAcum', hue='Año', palette=palette, linewidth=3)
plt.title('Crecimiento Acumulado de Ventas: Carrera Anual', weight='bold')
plt.xlabel('Día del Año')
plt.ylabel('Ventas Acumuladas ($)')
plt.legend(title='Año')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.savefig('historia_accionable_3_carrera_anual.png', bbox_inches='tight')
plt.close()

print("Se han generado 3 nuevos gráficos accionables: 'historia_accionable_1...', 'historia_accionable_2...' e 'historia_accionable_3...'.")